In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
%matplotlib inline

In [102]:
# Sorted by Precursor m/z
def sort_peaks2(batch_path):
    sample_df = pd.DataFrame()
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for f in range(num_files-1):
        standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
        standard['samples'] = os.listdir(batch_path)[f+1]
        sample_df = sample_df.append(standard)
    sorted_df = sample_df.sort_values(by=['Precursor m/z'])
    
    peak_num = 0
    listing = []
    for x in range(len(sorted_df)):
        if ((abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) <= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) <= 0.002)):
            listing.append(peak_num)
        else:
            peak_num = peak_num + 1
            listing.append(peak_num)
    sorted_df['unique peak'] = listing
    return sorted_df.groupby(['unique peak']).mean()

In [103]:
sorted_peaks = sort_peaks2('/mnt/c/Users/nozom/Desktop/mass-suite/example_data/batch/')
reference1 = sorted_peaks[['RT (min)','Precursor m/z', 'Area']]
reference1

,RT (min),Precursor m/z,Area
unique peak,,,
1,8.284537,100.039225,69996.908997
2,7.096592,100.076050,80018.282227
3,4.713334,100.111263,36846.741089
4,4.939933,100.111700,17319.328125
5,4.704889,100.111700,50668.457031
...,...,...,...
73105,17.049850,1487.098000,4589.567383
73106,17.104700,1488.096000,13429.444336
73107,0.636017,1499.038000,2858.513184


In [104]:
def stack(batch_path):
    all_samples = []
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for files in tqdm(range(num_files)):
        sample = os.listdir(batch_path)[files]
        sample_df = pd.read_csv(batch_path + sample, sep="\\t", usecols=['RT (min)', 'Precursor m/z', 'Area'], dtype=np.float32, engine='python')
        # Prefilter
        #sample_df = sample_df.drop(sample_df[(sample_df['Area'] > 5000) & (sample_df['RT (min)'] < 1.5) & (sample_df['S/N'] > 5)].index)
        all_samples += [sample_df]
    total_samples = pd.concat(all_samples)
    all_samples.clear()
    return total_samples

In [ ]:
"""This function works by using one .txt file as a reference in which
other files realigned to in terms of precursor and RT. """
RT_error = 0.10 # units of minutes, can be adjusted
alignment_df = pd.DataFrame()
standard_sample = os.listdir(batch_path)[0] # first sample
# reads .txt file into a dataframe
# standard_df = pd.read_csv(batch_path + standard_sample, sep="\\t", usecols=['RT (min)', 'Precursor m/z'], dtype= np.float32, engine='python')
standard_df = reference1
alignment_df['RT (min)'] = standard_df.iloc[:,0]
alignment_df['Precursor m/z'] = standard_df.iloc[:,1]
alignment_df['Sum RT (min)'] = 0.0
alignment_df['Sum Precursor m/z'] = 0.0
mz_error = 0.00002 # *standard_df.iloc[:,1] # 20 ppm
col_index = 4
total_samp = stack(batch_path)
num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))]) 
for files in range(num_files):
    sample = os.listdir(batch_path)[files]
    alignment_df[sample] = 0.0
    alignment_df[sample] = alignment_df[sample].astype('float32')
for row in tqdm(range(len(total_samp))):
    index_max = alignment_df.index.max()
    if row < range(len(total_samp))[-1]:
        if total_samp.index[row] < total_samp.index[row + 1]:
            overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                               (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                 & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                 & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
            if overlap[0].size:
                overlap = [overlap][0][0][0]
                # sets intensity value into slot where realignment was identified
                alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
            else:
                pass
        else:
            col_index += 1
            overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                               (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                 & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                 & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
            if overlap[0].size:
                overlap = [overlap][0][0][0]
                alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
            else:
                pass
    else:
        overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                            (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                             & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                             & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
        if overlap[0].size:
            overlap = [overlap][0][0][0]
            alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]

        else:
            pass

alignment_df = alignment_df.fillna(0)

alignment_df = alignment_df.drop(columns=['Sum RT (min)', 'Sum Precursor m/z'])

alignment_df

100%|██████████| 244935/244935 [22:16<00:00, 183.23it/s]


,RT (min),Precursor m/z,20180815_CEC_CAL-8-no4_MSpos_1.txt,20180815_CEC_CAL-8-no4_MSpos_2.txt,20180815_CEC_CAL-8-no4_MSpos_3.txt,20180815_CEC_CAL-8-no4_MSpos_4.txt,20180815_CEC_CAL-8-no4_MSpos_5.txt,20180815_CEC_CAL-8-no4_MSpos_6.txt,20180815_CEC_CAL-8-no4_MSpos_7.txt,20180815_CEC_CAL-8-no4_MSpos_8.txt,...,20180815_SR520-Creek_Mix6A_3.txt,20180815_SR520-Creek_Mix6B_1.txt,20180815_SR520-Creek_Mix6B_2.txt,20180815_SR520-Creek_Mix6B_3.txt,20180815_Swan-Creek-Dec_1.txt,20180815_Swan-Creek-Dec_2.txt,20180815_Swan-Creek-Dec_3.txt,20180815_Swan-Creek-May_1.txt,20180815_Swan-Creek-May_2.txt,20180815_Swan-Creek-May_3.txt
unique peak,,,,,,,,,,,,,,,,,,,,,
1,8.284537,100.039225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,7.096592,100.076050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,4.713334,100.111263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,4.939933,100.111700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50478.105469,0.0,0.0,0.0,0.0,0.0,42894.765625,0.0,0.0
5,4.704889,100.111700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73105,17.049850,1487.098000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
73106,17.104700,1488.096000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
73107,0.636017,1499.038000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [107]:
alignment_df.to_csv(batch_name + '.xlxs', header=True, index=False)